In [177]:
import pandas as pd
from pathlib import Path
import lxml
from lxml.etree import ElementTree,Element,SubElement
from lxml import etree

In [178]:
def set_xml_values_from_dict(el, a_dict):
    for k, v in a_dict.items():

        subel = el.find(k)
        if subel is not None:
            if v is not None:
                subel.text = str(v)
        else:
            subel = SubElement(el, k)
            subel.text = str(v)

In [179]:
def xml_container_element_to_value_dict(el, convert=True):
    a_dict = dict()
    for sub_el in el.getchildren():
        if len(sub_el.getchildren()) == 0:
            a_dict[sub_el.tag] = sub_el.text
            if convert is True:
                a_dict[sub_el.tag] = convert_xml_text(a_dict[sub_el.tag])
    if 'Index' in a_dict:
        a_dict['Index'] = str(a_dict['Index'])
    return a_dict

def convert_xml_text(text):
    """
    
    Parameters
    ----------
    text : str

    Returns
    -------
    resp : str,int,float
        converted text from xml in native python type.
    """

    if text is None:
        return text
    else:
        if (text.startswith(',') or text.startswith('.')) and text.endswith(tuple([str(el) for el in range(0, 10)])):
            text = '0' + text
        try:

            text_conv = int(text.replace(',', '.'))

        except ValueError:

            try:

                text_conv = float(text.replace(',', '.'))

            except ValueError:

                text_conv = text.replace('\n', '').replace('\t', '')

        return text_conv

In [180]:
path = Path("..")/"modele_donnee"

xml_validator = etree.XMLSchema(file=str((path/"DPEv2.xsd").absolute()))


df_mdd=pd.read_excel(path/'modele_donnee.xlsx')
df_mdd = df_mdd[['description','nom_variable','parent_xpath','type','condition','DPE NEUF']]

enum_table = pd.read_excel(path/'enum_tables.xlsx', sheet_name=None)

valeur_table = pd.read_excel(path/'valeur_tables.xlsx', sheet_name=None)
valeur_table['coef_masque_lointain_non_homogene']=valeur_table['coef_masque_lointain_non_homoge']
del valeur_table['coef_masque_lointain_non_homoge']
enums=[el for el in df_mdd.nom_variable.dropna().values.tolist() if el.startswith('enum')]
enums_without_id =[el for el in enums if not el.endswith('id')]
enums_without_id


enums=[el for el in df_mdd.nom_variable.dropna().values.tolist() if el.startswith('enum')]
enums_without_id =[el for el in enums if not el.endswith('id')]
enums_without_id

enums_dict = {k:v.set_index('id').lib.to_dict() for k,v in enum_table.items() if 'lib' in v and 'id' in v}



mdd_dict = dict()

for grp in df_mdd.parent_xpath.unique():
    mdd_dict[grp]={el:"" for el in df_mdd.loc[df_mdd.parent_xpath==grp].nom_variable.tolist()}


et=ElementTree(Element('dpe'))
dpe = et.getroot()



In [181]:
with open(r'D:\Cstbdata\MONWORKSPACE\Gitlab\py_3cl_dpe\py_3cl_dpe\json_sortie\cas_test_DHUP\cas_APT_1.json','r') as f:
    appt_1=json.load(f)

# dpe

In [182]:
dpe.set("version","1.0.0")

# administratif

In [183]:
administratif=SubElement(dpe,"administratif")

In [184]:
set_xml_values_from_dict(administratif,{
 'date_visite_diagnostiqueur': '2021-07-10',
 'date_etablissement_dpe':  '2021-07-10',
 'nom_proprietaire': 'Hubert Bonisseur de La Bath',
 'nom_proprietaire_installation_commune': "Syndic Saint D'hic",
    "enum_version_id":2,
 'enum_modele_dpe_id': 2})


In [185]:
diagnostiqueur=SubElement(administratif,"diagnostiqueur")

In [186]:
set_xml_values_from_dict(diagnostiqueur,{
 'usr_logiciel_id': '3',
 'version_logiciel': '3.2',
 'nom_diagnostiqueur': 'Lamar',
 'prenom_diagnostiqueur': 'Kendrick',
 'mail_diagnostiqueur': 'kendrick.lamar@diagnostiqueur.fr',
 'telephone_diagnostiqueur': '0607080910',
 'adresse_diagnostiqueur': '20 avenue de Ségur 75007 Paris',
 'numero_certification_diagnostiqueur': '????????????',
 'organisme_certificateur': 'Organisme Certificateur'})

In [187]:
geolocalisation=SubElement(administratif,"geolocalisation")

In [188]:
set_xml_values_from_dict(geolocalisation,{'invar_logement': '0151234567', 'idpar': '06163000CE8888',
                                          'immatriculation_copropriete': 'AA1234567'})

In [189]:
addr_vars=mdd_dict['dpe.administratif.geolocalisation.adresses.adresse_bien/adresse_proprietaire/adresse_proprietaire_installation_commune']

In [190]:
from geocoding_tools.addok_utils import get_addok_search

## adresses

In [191]:
adresses=SubElement(geolocalisation,"adresses")

### adresse propriétaire

In [192]:
addr = dict()
addr['adresse_brut']='3 rue roland martin'
addr['code_postal_brut']='94500'
addr['nom_commune_brut']="Champigny Sur Marne"
addr['label_brut']=' '.join([addr['adresse_brut'],addr['code_postal_brut'],addr['nom_commune_brut']])
addr['enum_statut_geocodage_ban_id']=1
addr['ban_date_appel']="2021-07-10"

addr.update({'compl_nom_residence': 'résidence les lilas',
 'compl_ref_batiment': 'A',
 'compl_etage_appartement': 0,
 'compl_ref_cage_escalier': 'A1',
 'compl_ref_logement': 'A12'})

addr_res_dict=get_addok_search(addr['label_brut'])[0]['properties']
addr_res_dict={'ban_'+k:v for k,v in addr_res_dict.items()}


for k,v in addr_vars.items():
    if k in addr_res_dict:
        addr[k]=addr_res_dict[k]

In [193]:
adresse_proprietaire=SubElement(adresses,"adresse_proprietaire")

In [194]:
set_xml_values_from_dict(adresse_proprietaire,addr)

In [195]:
addr = dict()

addr['adresse_brut']='116 avenue de Rosny'
addr['code_postal_brut']='93130'
addr['nom_commune_brut']="Noisy Le Sec"
addr['label_brut']=' '.join([addr['adresse_brut'],addr['code_postal_brut'],addr['nom_commune_brut']])
addr['enum_statut_geocodage_ban_id']=1
addr['ban_date_appel']="2021-07-10"

addr.update({'compl_nom_residence': 'résidence des iris',
 'compl_ref_batiment': 'B',
 'compl_etage_appartement':3,
 'compl_ref_logement': '40'})

addr_res_dict=get_addok_search(addr['label_brut'])[0]['properties']
addr_res_dict={'ban_'+k:v for k,v in addr_res_dict.items()}


for k,v in addr_vars.items():
    if k in addr_res_dict:
        addr[k]=addr_res_dict[k]

In [196]:
adresse_bien=SubElement(adresses,"adresse_bien")

In [197]:
set_xml_values_from_dict(adresse_bien,addr)

In [198]:
addr = dict()
addr['adresse_brut']='48 Avenue Marceau'
addr['code_postal_brut']='93130'
addr['nom_commune_brut']="Noisy-le-Sec"
addr['label_brut']=' '.join([addr['adresse_brut'],addr['code_postal_brut'],addr['nom_commune_brut']])
addr['enum_statut_geocodage_ban_id']=1
addr['ban_date_appel']="2021-07-10"



addr_res_dict=get_addok_search(addr['label_brut'])[0]['properties']
addr_res_dict={'ban_'+k:v for k,v in addr_res_dict.items()}


for k,v in addr_vars.items():
    if k in addr_res_dict:
        addr[k]=addr_res_dict[k]

In [199]:
addr

{'adresse_brut': '48 Avenue Marceau',
 'code_postal_brut': '93130',
 'nom_commune_brut': 'Noisy-le-Sec',
 'label_brut': '48 Avenue Marceau 93130 Noisy-le-Sec',
 'enum_statut_geocodage_ban_id': 1,
 'ban_date_appel': '2021-07-10',
 'ban_id': '93053_6120_00048',
 'ban_label': '48 Avenue Marceau 93130 Noisy-le-Sec',
 'ban_housenumber': '48',
 'ban_street': 'Avenue Marceau',
 'ban_citycode': '93053',
 'ban_postcode': '93130',
 'ban_city': 'Noisy-le-Sec',
 'ban_type': 'housenumber',
 'ban_score': 0.9741772727272727,
 'ban_x': 660505.01,
 'ban_y': 6865744.45}

In [200]:
adresse_proprietaire_installation_commune=SubElement(adresses,"adresse_proprietaire_installation_commune")
set_xml_values_from_dict(adresse_proprietaire_installation_commune,addr)

# identite logement

## carac general

In [201]:
logement_neuf=SubElement(dpe,"logement_neuf")


In [202]:
caracteristique_generale=SubElement(logement_neuf,"caracteristique_generale")


In [203]:

set_xml_values_from_dict(caracteristique_generale,{'annee_construction': 2012,
 'enum_periode_construction_id': 2,
 'enum_methode_application_dpe_log_id': 16,
 'surface_habitable_logement': 80.4,
'surface_habitable_immeuble': 402.7,
 'nombre_niveau_immeuble': 3,
 'nombre_niveau_logement': 1,
 'hsp': 2.8,
})

## meteo

In [204]:
meteo=SubElement(logement_neuf,"meteo")

set_xml_values_from_dict(meteo,{'enum_zone_climatique_id': 1,
 'enum_classe_altitude_id': 1,
 'batiment_materiaux_anciens': 0})

# enveloppe

In [205]:
enveloppe=SubElement(logement_neuf,"enveloppe")


## inertie

In [206]:
inertie=SubElement(enveloppe,"inertie")


In [207]:

set_xml_values_from_dict(inertie,{
 'enum_classe_inertie_id': 2})

## murs

In [208]:
mur_collection=SubElement(enveloppe,"mur_collection")


In [209]:
mur=SubElement(mur_collection,"mur")
de=SubElement(mur,"donnee_entree")
di=SubElement(mur,"donnee_intermediaire")


set_xml_values_from_dict(de,{ 'description': 'Mur extérieur sud',
'enum_type_adjacence_id': 1,
 'enum_orientation_id': 1,
 'surface_paroi_opaque': 71.5,
 'epaisseur_structure': 20,
 'enum_materiaux_structure_mur_id': 9,
 'enum_type_isolation_id': 3,
 'resistance_isolation': 4.7,
'epaisseur_isolation':20,
'enum_methode_saisie_u_id': 10})

set_xml_values_from_dict(di,{
'b': 1, 'umur': 0.19})

In [210]:
mur=SubElement(mur_collection,"mur")
de=SubElement(mur,"donnee_entree")
di=SubElement(mur,"donnee_intermediaire")


set_xml_values_from_dict(de,{ 'description': 'Mur extérieur nord',
'enum_type_adjacence_id': 1,
 'enum_orientation_id': 2,
 'surface_paroi_opaque': 71.5,
 'epaisseur_structure': 20,
 'enum_materiaux_structure_mur_id': 9,
 'enum_type_isolation_id': 3,
 'resistance_isolation': 4.7,
'epaisseur_isolation':20,
'enum_methode_saisie_u_id': 10})

set_xml_values_from_dict(di,{
'b': 1, 'umur': 0.19})

In [211]:
mur=SubElement(mur_collection,"mur")
de=SubElement(mur,"donnee_entree")
di=SubElement(mur,"donnee_intermediaire")


set_xml_values_from_dict(de,{ 'description': 'Mur extérieur est',
'enum_type_adjacence_id': 1,
 'enum_orientation_id': 3,
 'surface_paroi_opaque': 71.5,
 'epaisseur_structure': 20,
 'enum_materiaux_structure_mur_id': 9,
 'enum_type_isolation_id': 3,
 'resistance_isolation': 4.7,
'epaisseur_isolation':20,
'enum_methode_saisie_u_id': 10})

set_xml_values_from_dict(di,{
'b': 1, 'umur': 0.19})

In [212]:
mur=SubElement(mur_collection,"mur")
de=SubElement(mur,"donnee_entree")
di=SubElement(mur,"donnee_intermediaire")


set_xml_values_from_dict(de,{ 'description': 'Mur extérieur ouest',
'enum_type_adjacence_id': 1,
 'enum_orientation_id': 4,
 'surface_paroi_opaque': 71.5,
 'epaisseur_structure': 20,
 'enum_materiaux_structure_mur_id': 9,
 'enum_type_isolation_id': 3,
 'resistance_isolation': 4.7,
'epaisseur_isolation':20,
'enum_methode_saisie_u_id': 10})

set_xml_values_from_dict(di,{
'b': 1, 'umur': 0.19})

## planchers bas

In [213]:
plancher_bas_collection=SubElement(enveloppe,"plancher_bas_collection")
plancher_bas=SubElement(plancher_bas_collection,"plancher_bas")

de=SubElement(plancher_bas,"donnee_entree")
di=SubElement(plancher_bas,"donnee_intermediaire")


set_xml_values_from_dict(de,{ 'description': 'Plancher sur terre plein dalle beton',
'enum_type_adjacence_id': 5,
 'surface_paroi_opaque': 255.75,
 'enum_type_plancher_bas_id': 9,
 'enum_type_isolation_id': 3,
 'resistance_isolation': 4.65,
'epaisseur_isolation':10,
'enum_methode_saisie_u_id': 10})

set_xml_values_from_dict(di,{
'b':0.95, 'upb_final': 0.161})

## plancher haut

In [214]:
plancher_haut_collection=SubElement(enveloppe,"plancher_haut_collection")
plancher_haut=SubElement(plancher_haut_collection,"plancher_haut")



de=SubElement(plancher_haut,"donnee_entree")
di=SubElement(plancher_haut,"donnee_intermediaire")

set_xml_values_from_dict(de,{ 'description': 'Plancher sur terre plein dalle beton',
'enum_type_adjacence_id': 1,
 'surface_paroi_opaque': 255.75,
 'enum_type_plancher_haut_id': 1,
 'enum_type_isolation_id': 3,
 'resistance_isolation': 4.65,
'epaisseur_isolation':10,
'enum_methode_saisie_u_id': 10})

set_xml_values_from_dict(di,{
'b': 1, 'uph': 0.122})

## baie

In [215]:
baie_vitree_collection=SubElement(enveloppe,"baie_vitree_collection")


In [216]:
baie_vitree=SubElement(baie_vitree_collection,"baie_vitree")
de=SubElement(baie_vitree,"donnee_entree")
di=SubElement(baie_vitree,"donnee_intermediaire")


set_xml_values_from_dict(de,{

 'description': 'COUL.2 215/300 ALU KLINE UW1.3 SW0.55 VITRAGE 4/20/4 FE+A WARM EDGE',
 'enum_type_adjacence_id': 1,
 'surface_totale_baie':25 ,
 'nb_baie': 1,
 'enum_type_vitrage_id': 2,
 'enum_inclinaison_vitrage_id': 3,
 'enum_methode_saisie_perf_vitrage_id': 6,
 'enum_type_materiaux_menuiserie_id':6 ,
 'enum_type_baie_id': 5,
"epaisseur_lame":16,
    "enum_type_gaz_lame_id":2,
 'enum_orientation_id': 3,
})

set_xml_values_from_dict(di,{'b': 1,
 'u_menuiserie': 1.1,
 'sw': 0.06,
})

In [217]:
baie_vitree=SubElement(baie_vitree_collection,"baie_vitree")
de=SubElement(baie_vitree,"donnee_entree")
di=SubElement(baie_vitree,"donnee_intermediaire")


set_xml_values_from_dict(de,{

 'description': 'P.F1 215/100 ALU KLINE UW1.3 SW0.50 VITRAGE 4/16/4 FE+A WARM EDGE', 
 'enum_type_adjacence_id': 1,
 'surface_totale_baie':35,
 'enum_type_vitrage_id': 2,
 'enum_inclinaison_vitrage_id': 3,
 'enum_methode_saisie_perf_vitrage_id': 6,
 'enum_type_materiaux_menuiserie_id':6 ,
 'enum_type_baie_id': 8,
"epaisseur_lame":16,
    "enum_type_gaz_lame_id":2,
 'enum_orientation_id': 4,
})

set_xml_values_from_dict(di,{'b': 1,
 'u_menuiserie': 1.17,
 'sw': 0.06,
})

## porte

In [218]:
porte_collection=SubElement(enveloppe,"porte_collection")


In [219]:
porte=SubElement(porte_collection,"porte")
de=SubElement(porte,"donnee_entree")
di=SubElement(porte,"donnee_intermediaire")


set_xml_values_from_dict(de,{
 'description': 'porte',
 'enum_type_adjacence_id': 1,
 'surface_porte': 3.75,
 'enum_methode_saisie_uporte_id': 3,
 'enum_type_porte_id': 1                         
                         })

set_xml_values_from_dict(di,{'b': 0.45,
'uporte':3.5})

## ets

## PT

In [220]:
pont_thermique_collection=SubElement(enveloppe,"pont_thermique_collection")


pont_thermique=SubElement(pont_thermique_collection,"pont_thermique")
de=SubElement(pont_thermique,"donnee_entree")
di=SubElement(pont_thermique,"donnee_intermediaire")


set_xml_values_from_dict(de,{
 'l': 13.38,
})

set_xml_values_from_dict(di,{'k': 0.48})


pont_thermique=SubElement(pont_thermique_collection,"pont_thermique")
de=SubElement(pont_thermique,"donnee_entree")
di=SubElement(pont_thermique,"donnee_intermediaire")


set_xml_values_from_dict(de,{
 'l': 28.38,
})

set_xml_values_from_dict(di,{'k': 0.46})


pont_thermique=SubElement(pont_thermique_collection,"pont_thermique")
de=SubElement(pont_thermique,"donnee_entree")
di=SubElement(pont_thermique,"donnee_intermediaire")


set_xml_values_from_dict(de,{
 'l': 16.38,
})

set_xml_values_from_dict(di,{'k': 0.5})


# ventilation

In [221]:
ventilation_collection=SubElement(logement_neuf,"ventilation_collection")
ventilation=SubElement(ventilation_collection,"ventilation")
de=SubElement(ventilation,"donnee_entree")
di=SubElement(ventilation,"donnee_intermediaire")

In [222]:
set_xml_values_from_dict(de,{
 'enum_methode_saisie_q4pa_conv_id': 3,
 'enum_type_ventilation_id': 15,
 'ventilation_post_2012': 1})

set_xml_values_from_dict(di,{
 'q4pa_conv': 1,
 'conso_auxiliaire_ventilation': 2.3*402.7,
 'hperm':3.75,
 'hvent': 68.64})

# climatisation

In [223]:
climatisation_collection=SubElement(logement_neuf,"climatisation_collection")


# prod enr


In [224]:
#production_elec_enr=SubElement(logement_neuf,"production_elec_enr")


# ECS

##  installation ecs

In [225]:
installation_ecs_collection=SubElement(logement_neuf,"installation_ecs_collection")
installation_ecs=SubElement(installation_ecs_collection,"installation_ecs")

de=SubElement(installation_ecs,"donnee_entree")
di=SubElement(installation_ecs,"donnee_intermediaire")

set_xml_values_from_dict(de,{'description': 'installation simple ballon electrique',
                             "surface_habitable":80.4,
                             "nombre_logement":1,
 'enum_cfg_installation_ecs_id': 1,
 'enum_type_installation_id': 1,
})



## generateur ecs

In [226]:
generateur_ecs_collection=SubElement(installation_ecs,"generateur_ecs_collection")
generateur_ecs=SubElement(generateur_ecs_collection,"generateur_ecs")

de=SubElement(generateur_ecs,"donnee_entree")
di=SubElement(generateur_ecs,"donnee_intermediaire")


set_xml_values_from_dict(de,{'enum_type_generateur_ecs_id': 71,
 'ref_produit_generateur_ecs': 'Atlantic ZENEO',
 'description': 'ballon electrique',
 'enum_usage_generateur_id': 2,
 'enum_type_energie_id': 1,
 'enum_methode_saisie_carac_sys_id': 7,
 'enum_type_stockage_ecs_id': 2,
 'position_volume_chauffe': 1,
 'volume_stockage': 300})



# chauffage

## installation chauffage

In [227]:
installation_chauffage_collection=SubElement(logement_neuf,"installation_chauffage_collection")
installation_chauffage=SubElement(installation_chauffage_collection,"installation_chauffage")

de=SubElement(installation_chauffage,"donnee_entree")
di=SubElement(installation_chauffage,"donnee_intermediaire")

set_xml_values_from_dict(de,{'surface_chauffee': 80.4,
 'description': 'installation de chauffage individuelle',
 'enum_cfg_installation_ch_id': 1,
 'enum_type_installation_id': 1,
})

set_xml_values_from_dict(di,{'besoin_ch': 30.70*402.7,
})

## generateur chauffage

In [228]:
generateur_chauffage_collection=SubElement(installation_chauffage,"generateur_chauffage_collection")
generateur_chauffage=SubElement(generateur_chauffage_collection,"generateur_chauffage")

de=SubElement(generateur_chauffage,"donnee_entree")
di=SubElement(generateur_chauffage,"donnee_intermediaire")


set_xml_values_from_dict(de,{
 'enum_type_generateur_ch_id': 95,
 'description': 'convecteur electrique NFC',
 'enum_usage_generateur_id': 1,
 'enum_type_energie_id': 1,

})




## emetteur chauffage

In [229]:
emetteur_chauffage_collection=SubElement(installation_chauffage,"emetteur_chauffage_collection")
emetteur_chauffage=SubElement(emetteur_chauffage_collection,"emetteur_chauffage")

de=SubElement(emetteur_chauffage,"donnee_entree")


set_xml_values_from_dict(de,{
 'description': 'convecteur electrique NFC',
'enum_type_emission_distribution_id': 4,


})




# cle repartition chauffage

In [230]:
repartition_chauffage=SubElement(logement_neuf,"repartition_chauffage")

set_xml_values_from_dict(repartition_chauffage,{'surface_baie_nord': 0,
 'surface_baie_sud': 20,
 'surface_baie_est_ouest': 0,
 'surface_plancher_haut': 0,
 'surface_plancher_bas': 0,
    'surface_plancher_bas': 0,
    'surface_paroi_verticale_ext': 40,
'coef_ifc':0.7, 
'deperdition_totale_logement':42,
'deperdition_totale_batiment':42/0.14,
 'cle_repartition_ch': 0.14})


In [231]:
repartition_ecs=SubElement(logement_neuf,"repartition_ecs")

set_xml_values_from_dict(repartition_ecs,{
'besoin_ecs_logement':42,
'besoin_ecs_batiment':42/0.14,
 'cle_repartition_ecs': 0.14})


# sortie


In [232]:
sortie=SubElement(logement_neuf,"sortie")


## deperdition

In [233]:
deperdition=SubElement(sortie,"deperdition")
deper_dict={ 'hperm': 6.31785,
 'hvent': 58.344,
 'deperdition_renouvellement_air': 64.66,
 'deperdition_mur': 8.959999999999999+36.0+18.2,
 'deperdition_plancher_bas': 0,
 'deperdition_plancher_haut': 0,
 'deperdition_baie_vitree': 17.099999999999998+ 17.099999999999998+4.7,
'deperdition_porte':2.44,

 'deperdition_pont_thermique': 1.022+0.35+6.51+0.43+8.6+1.37+3.01,
}
deper_dict['deperdition_enveloppe']=0
for el in ['deperdition_mur', 'deperdition_plancher_bas', 'deperdition_plancher_haut', 'deperdition_baie_vitree', 'deperdition_pont_thermique']:
    deper_dict["deperdition_enveloppe"]+=deper_dict[el]
set_xml_values_from_dict(deperdition,deper_dict)


## apports

In [234]:
apport_et_besoin=SubElement(sortie,"apport_et_besoin")
set_xml_values_from_dict(apport_et_besoin,{
'besoin_ch': 8458.035143042967,
 'besoin_ecs': 1627.1638290072408,
 'besoin_fr': 0,
})



## conso

In [235]:
ef_conso=SubElement(sortie,"ef_conso")

set_xml_values_from_dict(ef_conso,{
 'conso_ch': 80.4*119.38543,
 'conso_ecs': 34.81*80.4,
    
 'conso_ch_depensier': 80.4*119.38543,
 'conso_ecs_depensier': 34.81*80.4,
 'conso_eclairage': 42,
 'conso_auxiliaire_ventilation': 0,
 'conso_totale_auxiliaire': 42,
 'conso_fr': 0,
     'conso_fr_depensier': 0,
 'conso_5_usages': 42,
 'conso_5_usages_m2': 50/2.58,
 })



In [236]:
ep_conso=SubElement(sortie,"ep_conso")

set_xml_values_from_dict(ep_conso,{
 'ep_conso_ch': 80.4*119.38543,
 'ep_conso_ecs_depensier': 34.81*80.4,
     'ep_conso_ch_depensier': 80.4*119.38543,
 'ep_conso_ecs': 34.81*80.4,
 'ep_conso_eclairage': 42,
 'ep_conso_auxiliaire_ventilation': 0,
 'ep_conso_totale_auxiliaire': 42,
 'ep_conso_fr': 0,
     'ep_conso_fr_depensier': 0,

 'ep_conso_5_usages': 42,
 'ep_conso_5_usages_m2': 50,
    'classe_conso_energie': 'A'
})

In [237]:
emission_ges=SubElement(sortie,"emission_ges")

set_xml_values_from_dict(emission_ges,{
 'emission_ges_ch': 80.4*119.38543,
 'emission_ges_ecs': 34.81*80.4,
     'emission_ges_ch_depensier': 80.4*119.38543,
 'emission_ges_ecs_depensier': 34.81*80.4,
 'emission_ges_eclairage': 42,
 'emission_ges_auxiliaire_ventilation': 0,
 'emission_ges_totale_auxiliaire': 42,
 'emission_ges_fr': 0,
     'emission_ges_fr_depensier': 0,
 'emission_ges_5_usages': 42,
 'emission_ges_5_usages_m2': 22.88,
'classe_emission_ges': 'A'
})

In [238]:
cout=SubElement(sortie,"cout")

set_xml_values_from_dict(cout,{
 'cout_ch': 80.4*119.38543,
 'cout_ecs': 34.81*80.4,
     'cout_ch_depensier': 80.4*119.38543,
 'cout_ecs_depensier': 34.81*80.4,
 'cout_eclairage': 42,
 'cout_auxiliaire_ventilation': 0,
 'cout_total_auxiliaire': 42,
 'cout_fr': 0,
     'cout_fr_depensier': 0,

 'cout_5_usages': 42,
})

In [239]:
production_electricite=SubElement(sortie,"production_electricite")
set_xml_values_from_dict(production_electricite,{
 'production_pv':  0,


})

In [240]:
sortie_par_energie_collection=SubElement(sortie,"sortie_par_energie_collection")
sortie_par_energie=SubElement(sortie_par_energie_collection,"sortie_par_energie")


In [241]:
set_xml_values_from_dict(sortie_par_energie,{'conso_ch':80.4*119.38543,
 'conso_ecs':  1.3*80.4*119.38543,
 'conso_5_usages':42,
 'enum_type_energie_id': 1,
 'emission_ges_ch': 42,
 'emission_ges_ecs': 42,
 'emission_ges_5_usages': 42,
 'cout_ch': 42,
 'cout_ecs': 42,
 'cout_5_usages': 42})

In [242]:
confort_ete=SubElement(sortie,"confort_ete")

set_xml_values_from_dict(confort_ete,{'isolation_toiture': 0,
 'protection_solaire_exterieure': 1,
 'aspect_traversant': 0,
 'brasseur_air': 0,
 'enum_indicateur_confort_ete_id': 3})

In [243]:
qualite_isolation=SubElement(sortie,"qualite_isolation")

set_xml_values_from_dict(qualite_isolation,{'ubat': 42,
 'qualite_isol_enveloppe': 1,
 'qualite_isol_mur': 1,
  'qualite_isol_plancher_haut_toit_terrasse': 1,
 'qualite_isol_plancher_bas': 1,
 'qualite_isol_menuiserie': 1})

# rset

In [244]:
import lxml

In [245]:
rset= SubElement(logement_neuf,'rset')

In [246]:
parser = lxml.etree.XMLParser(remove_blank_text=True)

et_rset=lxml.etree.parse('2522365.xml', parser).getroot()

In [247]:
rset.append(et_rset)

# descriptifs et fiches techniques

In [248]:
descriptif_enr_collection=SubElement(dpe,"descriptif_enr_collection")


In [249]:
descriptif_simplifie_collection=SubElement(dpe,"descriptif_simplifie_collection")


In [250]:
descriptif_simplifie=SubElement(descriptif_simplifie_collection,"descriptif_simplifie")
set_xml_values_from_dict(descriptif_simplifie,
{'description': "mur sud en briques pleines doubles avec lame d'air non isolé; mur est en briques pleines simples donnant sur une circulation",
 'enum_categorie_descriptif_simplifie_id':1})

descriptif_simplifie=SubElement(descriptif_simplifie_collection,"descriptif_simplifie")
set_xml_values_from_dict(descriptif_simplifie,{'description': 'plancher intermédiaire non déperditif donnant sur un autre logement adjacent',
                                                        'enum_categorie_descriptif_simplifie_id': 2})

descriptif_simplifie=SubElement(descriptif_simplifie_collection,"descriptif_simplifie")
set_xml_values_from_dict(descriptif_simplifie,{'description': 'plancher intermédiaire non déperditif donnant sur un autre logement adjacent',
                                                        'enum_categorie_descriptif_simplifie_id': 3})

descriptif_simplifie=SubElement(descriptif_simplifie_collection,"descriptif_simplifie")
set_xml_values_from_dict(descriptif_simplifie,
                         {'description': 'porte en bois pleine simple; Fenêtre battante bois simple vitrage orienté sud;Porte Fenêtre bois simple vitrage orienté sud',
                          'enum_categorie_descriptif_simplifie_id': 4})

descriptif_simplifie=SubElement(descriptif_simplifie_collection,"descriptif_simplifie")
set_xml_values_from_dict(descriptif_simplifie,{'description': 'convecteurs electriques anciens(système individuel)',
                                                        'enum_categorie_descriptif_simplifie_id': 5})

descriptif_simplifie=SubElement(descriptif_simplifie_collection,"descriptif_simplifie")
set_xml_values_from_dict(descriptif_simplifie,{'description': "ballon éléctrique vertical 300L", 
                                                        'enum_categorie_descriptif_simplifie_id': 6})

descriptif_simplifie=SubElement(descriptif_simplifie_collection,"descriptif_simplifie")
set_xml_values_from_dict(descriptif_simplifie,{'description': 'Sans Object', 'enum_categorie_descriptif_simplifie_id': 7})

descriptif_simplifie=SubElement(descriptif_simplifie_collection,"descriptif_simplifie")
set_xml_values_from_dict(descriptif_simplifie,{'description': 'ventilation naturelle par conduit', 'enum_categorie_descriptif_simplifie_id': 8})


In [251]:
fiche_technique_collection=SubElement(dpe,"fiche_technique_collection")

fiche_technique=SubElement(fiche_technique_collection,"fiche_technique")
set_xml_values_from_dict(fiche_technique,{'enum_categorie_fiche_technique_id': 1})


sous_fiche_technique_collection=SubElement(fiche_technique,"sous_fiche_technique_collection")


for description,enum_origine_donnee_id,valeur in [
  ('surface',2,'80m²'),
  ("type d'adjacence",2,'extérieur'),
('matériau mur',2,'briques pleines doubles'),
    ('état isolation',3,'non isolé'),
]:
    sous_fiche_technique=SubElement(sous_fiche_technique_collection,"sous_fiche_technique")
    set_xml_values_from_dict(sous_fiche_technique,
    {'description': description,
     'enum_origine_donnee_id': enum_origine_donnee_id,
    'valeur':valeur
    })


fiche_technique=SubElement(fiche_technique_collection,"fiche_technique")
set_xml_values_from_dict(fiche_technique,{'enum_categorie_fiche_technique_id': 1})
sous_fiche_technique_collection=SubElement(fiche_technique,"sous_fiche_technique_collection")

for description,enum_origine_donnee_id,valeur in [
  ('surface',2,'51m²'),
  ("type d'adjacence",2,'extérieur'),
('matériau mur',2,'bloc de béton plein'),
    ('état isolation',3,'isolé (ITI)'),
]:

    sous_fiche_technique=SubElement(sous_fiche_technique_collection,"sous_fiche_technique")
    set_xml_values_from_dict(sous_fiche_technique,
    {'description': description,
     'enum_origine_donnee_id': enum_origine_donnee_id,
    'valeur':valeur
    })



# fiche_technique=SubElement(fiche_technique_collection,"fiche_technique")
# set_xml_values_from_dict(fiche_technique,{'enum_categorie_fiche_technique_id': 1})


# sous_fiche_technique_collection=SubElement(fiche_technique,"sous_fiche_technique_collection")
# sous_fiche_technique=SubElement(sous_fiche_technique_collection,"sous_fiche_technique")
# set_xml_values_from_dict(sous_fiche_technique,{'description': "mur est en briques pleines simple non isolé donnant sur un immeuble a usage autre qu'habitation U = 3.5 W/m².K, b=0.2 S = 15m²",
#  'enum_origine_donnee_id': 2})



fiche_technique=SubElement(fiche_technique_collection,"fiche_technique")
set_xml_values_from_dict(fiche_technique,{'enum_categorie_fiche_technique_id': 4})

sous_fiche_technique_collection=SubElement(fiche_technique,"sous_fiche_technique_collection")
"Fenêtre battante orientée sud simple vitrage Ug = 5.8 W/m².K,Uw = 4.7 W/m².K, b=1"
for description,enum_origine_donnee_id,valeur in [
  ('surface',2,'5m²'),
  ("type d'adjacence",2,'extérieur'),
('type vitrage',2,'Double vitrage'),
]:
    sous_fiche_technique=SubElement(sous_fiche_technique_collection,"sous_fiche_technique")
    set_xml_values_from_dict(sous_fiche_technique,
    {'description': description,
     'enum_origine_donnee_id': enum_origine_donnee_id,
    'valeur':valeur
    })


# fiche_technique=SubElement(fiche_technique_collection,"fiche_technique")
# set_xml_values_from_dict(fiche_technique,{'enum_categorie_fiche_technique_id': 4})


# sous_fiche_technique_collection=SubElement(fiche_technique,"sous_fiche_technique_collection")
# sous_fiche_technique=SubElement(sous_fiche_technique_collection,"sous_fiche_technique")
# set_xml_values_from_dict(sous_fiche_technique,{'description': "Porte Fenêtre battante orientée sud simple vitrage équipé d'une jalousie accordéon Ug = 5.8 W/m².K,Uw = 4.7,Ujn = 3.8 W/m².K, b=1",
#            "detail_origine_donnee":"mesure du type de vitrage au vitromètre",                                    
#  'enum_origine_donnee_id': 2})


# fiche_technique=SubElement(fiche_technique_collection,"fiche_technique")
# set_xml_values_from_dict(fiche_technique,{'enum_categorie_fiche_technique_id': 4})


# sous_fiche_technique_collection=SubElement(fiche_technique,"sous_fiche_technique_collection")
# sous_fiche_technique=SubElement(sous_fiche_technique_collection,"sous_fiche_technique")
# set_xml_values_from_dict(sous_fiche_technique,{'description': "Porte Fenêtre battante orientée sud simple vitrage équipé d'une jalousie accordéon Ug = 5.8 W/m².K,Uw = 4.7,Ujn = 3.8 W/m².K, b=1",
#            "detail_origine_donnee":"mesure du type de vitrage au vitromètre",                                    
#  'enum_origine_donnee_id': 2})


# fiche_technique=SubElement(fiche_technique_collection,"fiche_technique")
# set_xml_values_from_dict(fiche_technique,{'enum_categorie_fiche_technique_id': 5})


# sous_fiche_technique_collection=SubElement(fiche_technique,"sous_fiche_technique_collection")
# sous_fiche_technique=SubElement(sous_fiche_technique_collection,"sous_fiche_technique")
# set_xml_values_from_dict(sous_fiche_technique,{'description': "Porte Simple en bois U = 3.5 W/m²/K",
# 'enum_origine_donnee_id': 2})



# fiche_technique=SubElement(fiche_technique_collection,"fiche_technique")
# set_xml_values_from_dict(fiche_technique,{'enum_categorie_fiche_technique_id': 6})


# sous_fiche_technique_collection=SubElement(fiche_technique,"sous_fiche_technique_collection")
# sous_fiche_technique=SubElement(sous_fiche_technique_collection,"sous_fiche_technique")
# set_xml_values_from_dict(sous_fiche_technique,{'description': "Pont thermique refend/mur l= 2.8 k=0.365 W/m/K",
# 'enum_origine_donnee_id': 2})


# fiche_technique=SubElement(fiche_technique_collection,"fiche_technique")
# set_xml_values_from_dict(fiche_technique,{'enum_categorie_fiche_technique_id': 6})


# sous_fiche_technique_collection=SubElement(fiche_technique,"sous_fiche_technique_collection")
# sous_fiche_technique=SubElement(sous_fiche_technique_collection,"sous_fiche_technique")
# set_xml_values_from_dict(sous_fiche_technique,{'description': "Pont thermique refend/mur l= 2.8 k=0.365 W/m/K",
# 'enum_origine_donnee_id': 2})

# fiche_technique=SubElement(fiche_technique_collection,"fiche_technique")
# set_xml_values_from_dict(fiche_technique,{'enum_categorie_fiche_technique_id': 6})


# sous_fiche_technique_collection=SubElement(fiche_technique,"sous_fiche_technique_collection")
# sous_fiche_technique=SubElement(sous_fiche_technique_collection,"sous_fiche_technique")
# set_xml_values_from_dict(sous_fiche_technique,{'description': "Pont thermique fenetre/mur l= 5 k=0.25 W/m/K",
# 'enum_origine_donnee_id': 2})

# fiche_technique=SubElement(fiche_technique_collection,"fiche_technique")
# set_xml_values_from_dict(fiche_technique,{'enum_categorie_fiche_technique_id': 6})

# sous_fiche_technique_collection=SubElement(fiche_technique,"sous_fiche_technique_collection")
# sous_fiche_technique=SubElement(sous_fiche_technique_collection,"sous_fiche_technique")
# set_xml_values_from_dict(sous_fiche_technique,{'description': "Pont thermique pf/mur l= 21 k=0.365 W/m/K",
# 'enum_origine_donnee_id': 2})

# fiche_technique=SubElement(fiche_technique_collection,"fiche_technique")
# set_xml_values_from_dict(fiche_technique,{'enum_categorie_fiche_technique_id': 6})

# sous_fiche_technique_collection=SubElement(fiche_technique,"sous_fiche_technique_collection")
# sous_fiche_technique=SubElement(sous_fiche_technique_collection,"sous_fiche_technique")
# set_xml_values_from_dict(sous_fiche_technique,{'description': 'pont thermique Plancher intermediaire/mur l= 16 k=0.43 W/m/K',
# 'enum_origine_donnee_id': 2})



# fiche_technique=SubElement(fiche_technique_collection,"fiche_technique")
# set_xml_values_from_dict(fiche_technique,{'enum_categorie_fiche_technique_id': 7})

# sous_fiche_technique_collection=SubElement(fiche_technique,"sous_fiche_technique_collection")
# sous_fiche_technique=SubElement(sous_fiche_technique_collection,"sous_fiche_technique")
# set_xml_values_from_dict(sous_fiche_technique,{'description': 'convecteurs electriques anciens Rg=1 Re=0.95 Rr=0.9 Rd=1',
# 'enum_origine_donnee_id': 2})

# fiche_technique=SubElement(fiche_technique_collection,"fiche_technique")
# set_xml_values_from_dict(fiche_technique,{'enum_categorie_fiche_technique_id': 8})

# sous_fiche_technique_collection=SubElement(fiche_technique,"sous_fiche_technique_collection")
# sous_fiche_technique=SubElement(sous_fiche_technique_collection,"sous_fiche_technique")
# set_xml_values_from_dict(sous_fiche_technique,{'description': 'ballon electrique vertical atlantic zeneo 300L Rg=1 Rs=0.64 Rd=0.93',
# 'enum_origine_donnee_id': 3})

# fiche_technique=SubElement(fiche_technique_collection,"fiche_technique")
# set_xml_values_from_dict(fiche_technique,{'enum_categorie_fiche_technique_id': 10})

# sous_fiche_technique_collection=SubElement(fiche_technique,"sous_fiche_technique_collection")
# sous_fiche_technique=SubElement(sous_fiche_technique_collection,"sous_fiche_technique")
# set_xml_values_from_dict(sous_fiche_technique,{'description': 'Ventilation naturelle par conduit ',
# 'enum_origine_donnee_id': 2})

# justificatifs

In [252]:
justificatif_collection=SubElement(dpe,"justificatif_collection")
justificatif=SubElement(justificatif_collection,"justificatif")
set_xml_values_from_dict(justificatif,
{'description': "notice technique du ballon d'eau chaude atlantic zeneo", 'enum_type_justificatif_id':10 })

# gestes entretien

In [253]:
descriptif_geste_entretien_collection=SubElement(dpe,"descriptif_geste_entretien_collection")
descriptif_geste_entretien=SubElement(descriptif_geste_entretien_collection,"descriptif_geste_entretien")
set_xml_values_from_dict(descriptif_geste_entretien,
{'enum_picto_geste_entretien_id': 9,
 'description': "detarter et vidanger le ballon d'eau chaude tous les 5 ans",
 'categorie_geste_entretien': 'entretien du chauffe-eau'})

In [254]:
len('etat_initial_consommation_energie_estim_mean_1')

46

# valid

In [255]:
et.write(str((path/'exemples_metier'/'cas_test_appt_2_neuf_valid.xml')), pretty_print=True, xml_declaration=True, encoding='utf-8')

In [256]:


is_valid = xml_validator.validate(et)

In [257]:
xml_validator.error_log

In [258]:
is_valid

True

# variante light

In [259]:
el=dpe.getchildren()[1]

In [260]:
namespaces = {'xs':'http://www.w3.org/2001/XMLSchema'}

In [261]:
logement_neuf=dpe.xpath('.//logement_neuf')[0]

for el in ['enveloppe','ventilation_collection',
           'installation_chauffage_collection',
           'installation_ecs_collection','climatisation_collection',
           'production_elec_enr','deperdition','apport_et_besoin']:
    el = list(logement_neuf.iterfind(f'.//{el}',namespaces=namespaces))
    if len(el)>0:
        el = el[0]
        print(el)
        el.getparent().remove(el)

<Element enveloppe at 0x1890ff74ac0>
<Element ventilation_collection at 0x189100839c0>
<Element installation_chauffage_collection at 0x1891007c5c0>
<Element installation_ecs_collection at 0x1890ff66c00>
<Element climatisation_collection at 0x189101db8c0>
<Element deperdition at 0x1891372a0c0>
<Element apport_et_besoin at 0x18913641740>


In [262]:
is_valid = xml_validator.validate(et)

In [263]:
is_valid

True

In [264]:
import requests

dpe.find('*enum_version_id').text='2'

r=requests.post('http://localhost:5000/controle_coherence',data=etree.tostring(dpe))

r.json()

{'erreur_logiciel': [],
 'erreur_saisie': [],
 'message_principal': 'CONTROLE DE COHERENCE DU MODELE EFFECTUE SUR DPE NEUF.',
 'notification': [],
 'validation_xsd': {'error_log': [''], 'valid': True},
 'warning_logiciel': [],
 'warning_saisie': []}

In [265]:
xml_validator.error_log

In [266]:
import datetime

In [267]:
import time

In [268]:
time.localtime()

time.struct_time(tm_year=2021, tm_mon=5, tm_mday=27, tm_hour=23, tm_min=26, tm_sec=3, tm_wday=3, tm_yday=147, tm_isdst=1)

In [269]:
is_valid 

True

In [270]:
et.write(str((path/'exemples_metier'/'cas_test_appt_2_neuf_light_valid.xml')), pretty_print=True, xml_declaration=True, encoding='utf-8')

In [271]:
import requests

dpe.find('*enum_version_id').text='2'

r=requests.post('http://localhost:5000/controle_coherence',data=etree.tostring(dpe))

r.json()

{'erreur_logiciel': [],
 'erreur_saisie': [],
 'message_principal': 'CONTROLE DE COHERENCE DU MODELE EFFECTUE SUR DPE NEUF.',
 'notification': [],
 'validation_xsd': {'error_log': [''], 'valid': True},
 'warning_logiciel': [],
 'warning_saisie': []}